<a href="https://colab.research.google.com/github/UmarIgan/Machine-Learning/blob/master/Hosting_Llama2_Code_7B_On_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Llama 2 Code Streamlit app with Free GPU via Google Collab**
- motivation: https://huggingface.co/blog/codellama

---

- First make sure you have gpu on on you colab
- Install requirements for running  model and streamlit

- Install requirements for running  model and streamlit


In [1]:
!pip install streamlit python-multipart transformers accelerate bitsandbytes pyngrok==4.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.3 MB/s eta 0:00:00

writing streamlit app file, %%writefile app.py command will help you with creating app file

In [2]:
%%writefile app.py
import streamlit as st
from threading import Thread
from typing import Iterator
import torch
import accelerate
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer


model_id = 'codellama/CodeLlama-7b-Instruct-hf'
cache_dir = 'model'


if torch.cuda.is_available():
    config = AutoConfig.from_pretrained(model_id,cache_dir=cache_dir)
    config.pretraining_tp = 1
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        config=config,
        torch_dtype=torch.float16,
        load_in_4bit=True,
        device_map='auto',
        use_safetensors=False,
    )
else:
    model = None
tokenizer = AutoTokenizer.from_pretrained(model_id,cache_dir=cache_dir)


DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant with a deep knowledge of code and software design.
Always answer as helpfully as possible, while being safe. Your answers should not include any harmful,
unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
\n\nIf a question does not make any sense, or is not factually coherent,
explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.\
"""
MAX_MAX_NEW_TOKENS = 4096
DEFAULT_MAX_NEW_TOKENS = 1024
MAX_INPUT_TOKEN_LENGTH = 4000



def get_prompt(message: str, chat_history: list[tuple[str, str]],
               system_prompt: str) -> str:
    texts = [f'<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n']
    # The first user input is _not_ stripped
    do_strip = False
    for user_input, response in chat_history:
        user_input = user_input.strip() if do_strip else user_input
        do_strip = True
        texts.append(f'{user_input} [/INST] {response.strip()} </s><s>[INST] ')
    message = message.strip() if do_strip else message
    texts.append(f'{message} [/INST]')
    return ''.join(texts)

def get_input_token_length(message: str, chat_history: list[tuple[str, str]], system_prompt: str) -> int:
    prompt = get_prompt(message, chat_history, system_prompt)
    input_ids = tokenizer([prompt], return_tensors='np', add_special_tokens=False)['input_ids']
    return input_ids.shape[-1]


def run(message: str,
        chat_history: list[tuple[str, str]],
        system_prompt: str,
        max_new_tokens: int = 1024,
        temperature: float = 0.1,
        top_p: float = 0.9,
        top_k: int = 50) -> Iterator[str]:
    prompt = get_prompt(message, chat_history, system_prompt)
    inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')

    streamer = TextIteratorStreamer(tokenizer,
                                    timeout=10.,
                                    skip_prompt=True,
                                    skip_special_tokens=True)
    generate_kwargs = dict(
        inputs,
        streamer=streamer,
        do_sample=True,
        max_new_tokens = 1024,
        temperature = 0.1,
        top_p = 0.9,
        top_k = 50,
        num_beams=1)
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()

    outputs = []
    for text in streamer:
        outputs.append(text)
        yield ''.join(outputs)

def main():
    st.title("Code Llama 7B Chat on Colab GPU by Umar IGAN")
    st.markdown("""
    This Space demonstrates model [CodeLlama-7b-Instruct](https://huggingface.co/codellama/CodeLlama-7b-Instruct-hf) by Meta, a Code Llama model with 7B parameters fine-tuned for chat instructions and specialized on code tasks.
    I build this on colab in a way to use your colab gpu on this interface.
    """)

    message = st.text_area("Type a message...")
    system_prompt = st.text_area("System prompt", value=DEFAULT_SYSTEM_PROMPT)

    if st.button("Submit"):
        history = []
        input_token_length = get_input_token_length(message, history, system_prompt)
        if input_token_length > MAX_INPUT_TOKEN_LENGTH:
            st.error(f'The accumulated input is too long ({input_token_length} > {MAX_INPUT_TOKEN_LENGTH}). Clear your chat history and try again.')
            return

        output_elem = st.empty()  # Create an empty element for dynamic text update

        def update_output(response):
            history.append((message, response))
            output_elem.text(response)  # Update the element's content with the response

        generator = run(message, history, system_prompt, max_new_tokens, temperature, top_p, top_k)
        for response in generator:
            #st.write(response)  # Display response in the sidebar as well
            update_output(response)  # Update the dynamic text element with the response


    if st.button("Clear"):
        st.text_area("Type a message...", value="")

    if st.button("Retry"):
        pass  # Implement the retry logic here

if __name__ == "__main__":
    main()


Writing app.py


we added log file to track download process of model files.

You can get a public url running codes below.



In [3]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.68s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [4]:
!streamlit run app.py &>/content/logs.txt &

Get your ip address

In [5]:
!wget -q -O - ipv4.icanhazip.com

34.30.29.231


In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.008s
your url is: https://ready-emus-attack.loca.lt
